# Strategy

Choose stocks which are underperforming the S&P 500 according to their variation from the OLS regression between the stock and SPY. Use a pair of thresholds for the z-score and determine the cumulative returns.

In [1]:
import yfinance as yf
import pandas as pd

# Read and print the stock tickers that make up the S&P 500 - from https://gist.github.com/quantra-go-algo/ac5180bf164a7894f70969fa563627b2

tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
look_back_window = 10 * 7
data_look_back = '6mo'

tickers

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...
498,XYL,Xylem Inc.,Industrials,Industrial Machinery & Supplies & Components,"White Plains, New York",2011-11-01,1524472,2011
499,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
500,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
501,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927


In [5]:
sp_500_data = yf.download(tickers["Symbol"].to_list(), period=data_look_back, interval='1h',
                          auto_adjust=True)
sp_500_data

[*********************100%***********************]  503 of 503 completed

2 Failed downloads:
['BRK.B']: YFPricesMissingError('possibly delisted; no price data found  (period=6mo) (Yahoo error = "No data found, symbol may be delisted")')
['BF.B']: YFPricesMissingError('possibly delisted; no price data found  (period=6mo)')


Price                     Adj Close             Close                          \
Ticker                         BF.B BRK.B           A        AAPL        ABBV   
2024-09-09 13:30:00+00:00       NaN   NaN  138.809998  218.875000  194.949997   
2024-09-09 14:30:00+00:00       NaN   NaN  138.389999  218.960007  195.720001   
2024-09-09 15:30:00+00:00       NaN   NaN  139.110001  220.203903  196.100006   
2024-09-09 16:30:00+00:00       NaN   NaN  138.824997  220.839996  196.156204   
2024-09-09 17:30:00+00:00       NaN   NaN  138.029999  217.169998  196.300003   
...                             ...   ...         ...         ...         ...   
2025-03-07 16:30:00+00:00       NaN   NaN  125.394997  239.289993  213.319901   
2025-03-07 17:30:00+00:00       NaN   NaN  126.724998  240.414398  214.373001   
2025-03-07 18:30:00+00:00       NaN   NaN  127.269997  239.649994  213.949997   
2025-03-07 19:30:00+00:00       NaN   NaN  127.495003  238.811401  214.669998   
2025-03-07 20:30:00+00:00       NaN   NaN  126.669998  239.070007  214.264999   

Price                                                                      \
Ticker                           ABNB         ABT        ACGL         ACN   
2024-09-09 13:30:00+00:00  116.870003  115.510002  111.415001  339.950012   
2024-09-09 14:30:00+00:00  116.184998  115.940002  111.589996  339.445007   
2024-09-09 15:30:00+00:00  117.105003  116.349998  112.150002  341.579987   
2024-09-09 16:30:00+00:00  117.629898  116.525002  112.264999  341.899994   
2024-09-09 17:30:00+00:00  117.489998  116.449997  112.129997  340.698395   
...                               ...         ...         ...         ...   
2025-03-07 16:30:00+00:00  130.949997  136.000000   90.820000  337.309998   
2025-03-07 17:30:00+00:00  132.479996  136.824997   91.230003  340.454987   
2025-03-07 18:30:00+00:00  134.354996  137.274994   91.334999  341.720001   
2025-03-07 19:30:00+00:00  133.949997  137.225006   91.430000  340.929993   
2025-03-07 20:30:00+00:00  134.389999  137.130005   91.040001  342.179993   

Price                                  ...   Volume                      \
Ticker                           ADBE  ...      WTW        WY      WYNN   
2024-09-09 13:30:00+00:00  570.489990  ...      0.0  595791.0  194470.0   
2024-09-09 14:30:00+00:00  568.205017  ...  30333.0  243424.0  162059.0   
2024-09-09 15:30:00+00:00  572.159973  ...  25134.0  207791.0  126717.0   
2024-09-09 16:30:00+00:00  573.929993  ...  33480.0  177430.0  242000.0   
2024-09-09 17:30:00+00:00  567.549988  ...  68360.0  183226.0  110390.0   
...                               ...  ...      ...       ...       ...   
2025-03-07 16:30:00+00:00  443.290009  ...  52973.0  514523.0  218533.0   
2025-03-07 17:30:00+00:00  446.209991  ...  53894.0  351777.0  168252.0   
2025-03-07 18:30:00+00:00  450.599915  ...  44586.0  409864.0  244072.0   
2025-03-07 19:30:00+00:00  450.760010  ...  33911.0  584731.0  222922.0   
2025-03-07 20:30:00+00:00  449.380005  ...  80593.0  649997.0  311493.0   

Price                                                                          \
Ticker                           XEL        XOM       XYL       YUM       ZBH   
2024-09-09 13:30:00+00:00   262360.0  3480333.0  231777.0  211356.0  245707.0   
2024-09-09 14:30:00+00:00   147319.0  3017606.0   74784.0  151969.0  188845.0   
2024-09-09 15:30:00+00:00   261741.0  2304761.0   62441.0  117211.0  151348.0   
2024-09-09 16:30:00+00:00   204356.0  1901616.0   65413.0  158757.0  111291.0   
2024-09-09 17:30:00+00:00   146870.0  1580271.0   77682.0  241970.0  157687.0   
...                              ...        ...       ...       ...       ...   
2025-03-07 16:30:00+00:00   392459.0  1978236.0  109684.0  295787.0  172616.0   
2025-03-07 17:30:00+00:00   269254.0  1497116.0  188180.0  275950.0  169847.0   
2025-03-07 18:30:00+00:00   254102.0  1341543.0   84561.0  333226.0  242512.0   
2025-03-07 19:30:00+00:00   369855.0  1262282.0  100876.0  477254.0  323429.0

In [7]:
sp_500_close_data = sp_500_data["Close"].dropna(axis=1, how="all").ffill()
sp_500_close_data

eastern_indices = sp_500_close_data.index.tz_convert('US/Eastern')
eastern_indices

DatetimeIndex(['2024-09-09 09:30:00-04:00', '2024-09-09 10:30:00-04:00',
               '2024-09-09 11:30:00-04:00', '2024-09-09 12:30:00-04:00',
               '2024-09-09 13:30:00-04:00', '2024-09-09 14:30:00-04:00',
               '2024-09-09 15:30:00-04:00', '2024-09-10 09:30:00-04:00',
               '2024-09-10 10:30:00-04:00', '2024-09-10 11:30:00-04:00',
               ...
               '2025-03-06 13:30:00-05:00', '2025-03-06 14:30:00-05:00',
               '2025-03-06 15:30:00-05:00', '2025-03-07 09:30:00-05:00',
               '2025-03-07 10:30:00-05:00', '2025-03-07 11:30:00-05:00',
               '2025-03-07 12:30:00-05:00', '2025-03-07 13:30:00-05:00',
               '2025-03-07 14:30:00-05:00', '2025-03-07 15:30:00-05:00'],
              dtype='datetime64[ns, US/Eastern]', length=862, freq=None)

In [8]:
sp_500_close_data.set_index(eastern_indices, inplace=True)

In [9]:
sp_500_close_data.index.hour

Index([ 9, 10, 11, 12, 13, 14, 15,  9, 10, 11,
       ...
       13, 14, 15,  9, 10, 11, 12, 13, 14, 15],
      dtype='int32', length=862)

In [11]:
tickers["GICS Sector"].unique()

array(['Industrials', 'Health Care', 'Information Technology',
       'Utilities', 'Financials', 'Materials', 'Consumer Discretionary',
       'Real Estate', 'Communication Services', 'Consumer Staples',
       'Energy'], dtype=object)

In [12]:
sectors_to_tickers = {
    "Industrials": "SP500-20",
    "Health Care": "SP500-35",
}

yf.download(["SP500-20", "SP500-35"], period=data_look_back, interval='1h',
                          auto_adjust=True)

[*********************100%***********************]  2 of 2 completed

2 Failed downloads:
['SP500-35', 'SP500-20']: YFPricesMissingError('possibly delisted; no price data found  (period=6mo) (Yahoo error = "No data found, symbol may be delisted")')


Empty DataFrame
Columns: [(Adj Close, SP500-20), (Adj Close, SP500-35), (Close, SP500-20), (Close, SP500-35), (High, SP500-20), (High, SP500-35), (Low, SP500-20), (Low, SP500-35), (Open, SP500-20), (Open, SP500-35), (Volume, SP500-20), (Volume, SP500-35)]
Index: []

In [24]:
wednesday_first_hour_closing_px = sp_500_close_data[
    (sp_500_close_data.index.weekday == 2) & (sp_500_close_data.index.hour == 9)]
wednesday_first_hour_closing_px

Ticker,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
2024-08-21 09:30:00-04:00,139.904999,227.225006,196.320007,117.480003,110.970001,103.379997,331.769989,562.690002,228.460007,59.070000,...,281.899994,30.000000,77.269997,59.865002,114.949997,134.634995,137.983795,111.434998,345.140015,183.255005
2024-08-28 09:30:00-04:00,141.949997,229.639999,194.160004,116.279999,112.735001,110.823502,342.214996,568.994995,229.300003,60.630001,...,290.762512,30.684999,75.250000,60.814999,116.500000,136.429993,135.414993,114.639999,345.850006,184.425003
2024-09-04 09:30:00-04:00,138.580002,218.634995,197.789993,115.055000,115.040001,113.665001,343.959991,573.880005,221.440002,61.075001,...,294.045013,30.500000,77.199997,62.930000,114.900002,129.994995,134.330002,116.070000,329.640015,182.399994
2024-09-11 09:30:00-04:00,134.975006,219.250000,194.169998,113.227501,114.907402,109.269997,342.910004,563.405029,216.100006,58.455002,...,283.519989,30.340000,74.974998,63.209999,108.535004,126.769997,131.289993,103.510002,328.769989,186.884995
2024-09-18 09:30:00-04:00,138.429993,220.735001,193.860001,123.320000,115.714996,113.709999,334.320007,508.700012,224.729996,60.369999,...,293.029999,33.099998,78.860001,64.540001,113.900002,132.979996,132.789993,105.730003,349.500000,195.320007
2024-09-25 09:30:00-04:00,141.520004,226.384995,192.740005,130.730698,112.320000,112.820000,337.940002,521.690002,226.490005,58.980000,...,290.679993,33.630001,84.010002,64.309998,115.269997,133.770004,132.570007,107.169998,364.839996,194.149994
2024-10-02 09:30:00-04:00,146.440002,225.419998,195.925003,125.529999,113.050003,113.440002,354.459991,509.829895,228.289993,59.380001,...,298.894989,33.630001,100.190002,65.209999,121.769997,134.529907,139.119995,106.040001,367.364990,193.720001
2024-10-09 09:30:00-04:00,144.470001,227.100006,192.360001,133.434998,114.709999,111.235001,362.406006,493.160004,230.520004,57.450001,...,293.160004,32.900002,103.260002,62.630001,121.400002,135.070007,135.080002,101.620003,372.265015,187.229996
2024-10-16 09:30:00-04:00,140.990005,230.729706,189.660004,135.934998,119.059998,108.650002,370.279999,502.704987,229.865005,58.000099,...,292.470001,33.345001,100.739998,63.790001,121.000000,137.309479,133.300003,105.224998,378.799988,195.009995
2024-10-23 09:30:00-04:00,132.350006,234.720001,188.229996,134.789993,116.370003,106.805000,370.575012,489.179993,228.300003,55.630001,...,290.720001,31.709999,98.660004,63.945000,120.419998,132.455994,134.673706,104.139999,370.535004,188.445007


In [25]:
sp_500_close_data

Ticker,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
2024-08-15 09:30:00-04:00,138.860001,224.789993,191.699997,116.885002,110.349998,101.110001,323.040009,556.719971,220.759995,58.410900,...,279.989990,30.090000,76.114998,58.665001,118.559998,132.869995,137.000000,109.070000,341.589294,183.740005
2024-08-15 10:30:00-04:00,139.009995,224.005005,192.184998,117.019997,110.379997,101.565002,323.959991,553.739990,221.809998,59.029999,...,280.630005,30.309999,76.775002,58.709999,119.540001,133.080002,137.514999,109.809998,342.989990,183.639999
2024-08-15 11:30:00-04:00,139.160004,224.994995,192.490005,117.849998,110.489998,101.570000,324.559998,555.090027,223.899994,59.020000,...,280.565002,30.360001,76.580002,58.689999,119.239700,133.475006,137.850006,109.964996,343.820007,184.100006
2024-08-15 12:30:00-04:00,139.210007,224.544998,191.889999,117.890602,110.540001,101.739998,324.515015,555.150024,222.699997,58.900002,...,280.339996,30.334999,76.440002,58.759998,119.117996,133.179993,138.029694,109.779999,343.779999,183.899994
2024-08-15 13:30:00-04:00,139.399994,225.070007,192.095001,118.610001,110.760002,101.690002,324.079987,555.250000,223.580002,58.865002,...,280.345001,30.219999,76.580002,58.955002,119.074501,133.229996,137.744995,109.529999,344.000000,183.860001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-14 11:30:00-05:00,135.789993,243.815201,194.369995,162.369995,130.899994,88.747200,388.325012,459.984985,213.535004,46.009998,...,323.274994,29.754999,88.724998,69.019997,108.970001,130.014999,148.005005,101.339996,318.339996,159.339996
2025-02-14 12:30:00-05:00,135.660004,243.999893,193.705704,161.760803,131.029999,88.670097,388.149994,459.980011,213.725006,46.070000,...,323.709991,29.705000,89.209999,68.919998,108.800003,130.029999,147.910004,101.650002,317.785004,159.880005
2025-02-14 13:30:00-05:00,135.169998,244.149994,193.610001,161.210007,130.945007,88.559998,387.929993,461.489990,213.395004,46.110001,...,322.820007,29.620001,89.080002,68.684998,108.495003,129.779999,148.205002,101.500000,317.515015,157.960007
2025-02-14 14:30:00-05:00,134.830002,244.100006,193.479996,161.490005,130.619995,88.275002,388.010010,459.144989,213.455002,46.139999,...,321.565002,29.650000,88.589996,68.505402,108.370003,129.393600,147.910004,101.044998,317.970001,157.360001
